# Forward
In the notebook we clean and merge the data set. 
The data set is anonymized.
The are 4 file that we clean drop unwanted information and merge into one data base:

1) Payment tracking 2017 to 2018

2) Payment tracking 2018 to 2019

3) contractor title

4) Tasks reports

In [157]:
import pandas as pd
import numpy as np
import datetime
import re
from scipy import stats

In [158]:
pay2017 = pd.read_csv("../Data_to_clean/payend_2017_beg_2018_anon.csv")
pay18_19 = pd.read_csv("../Data_to_clean/pay18-19_anon.csv")
title = pd.read_csv("../Data_to_clean/cont_info_anon.csv")
time_entries = pd.read_csv("../Data_to_clean/time_entries_anon.csv")

## cleaning and merging the payment tracking


In [159]:
pay2017.head()

,Unnamed: 0,Month,Name,Hourly,Total,$
0,0,September 2017,UI32,$31.00,66.33,"$2,056.33"
1,1,September 2017,UI13,NaN,179.78,"$3,700.00"
2,2,September 2017,UI24,$17.00,20.00,$340.00
3,3,September 2017,UI28,$18.00,24.93,$448.80
4,4,September 2017,UI28,$18.00,6.53,$117.60


In [160]:
pay18_19.head()

,Unnamed: 0,Month,Name,Duration,Decimal,Type,Total
0,0,2018-02-01,UI10,90:54:15,90.90,Hourly,2272.60
1,1,2018-02-01,NaN,26:18:14,26.30,Hourly,604.99
2,2,2018-02-01,UI13,164:45:35,164.76,Monthly,3700.00
3,3,2018-02-01,UI14,59:00:30,59.01,Hourly,3180.17
4,4,2018-02-01,UI15,72:30:00,72.50,Hourly,1450.00


In [161]:
pay2017.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124 entries, 0 to 123
Data columns (total 6 columns):
Unnamed: 0    124 non-null int64
Month         124 non-null object
Name          114 non-null object
Hourly        123 non-null object
Total         123 non-null float64
$             124 non-null object
dtypes: float64(1), int64(1), object(4)
memory usage: 5.9+ KB


In [162]:
pay18_19.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 488 entries, 0 to 487
Data columns (total 7 columns):
Unnamed: 0    488 non-null int64
Month         488 non-null object
Name          475 non-null object
Duration      488 non-null object
Decimal       488 non-null float64
Type          488 non-null object
Total         488 non-null float64
dtypes: float64(2), int64(1), object(4)
memory usage: 26.8+ KB


### 1)  preping pay2017 to fit pay18_19

In [163]:
#Changing to information to floats from string
pay2017['$'] = pay2017['$'].str.replace("$", "").str.replace(",", "").astype(float)
pay2017['Hourly'] = pay2017['Hourly'].str.replace("$", "").str.replace(",", "").astype(float)

In [164]:
#droping rows with no UIs
pay2017.dropna(subset=["Name"], inplace=True)

In [165]:
#grouping by to recieve one amount per UI in a given month
pay2017=pay2017.groupby(['Month','Name'], as_index=False).agg({
    "Total" : 'sum',
    "$" : 'sum',
    'Hourly': 'sum'
})

In [166]:
pay2017.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57 entries, 0 to 56
Data columns (total 5 columns):
Month     57 non-null object
Name      57 non-null object
Total     57 non-null float64
$         57 non-null float64
Hourly    57 non-null float64
dtypes: float64(3), object(2)
memory usage: 2.7+ KB


In [167]:
# generating a Type column based on the Hourly column (with information from the $ column) 
pay2017.Hourly.fillna(0) 
pay2017.loc[pay2017["Hourly"]== 0]

,Month,Name,Total,$,Hourly
0,December 2017,UI13,157.03,3700.0,0.0
28,November 2017,UI13,188.35,3700.0,0.0
40,October 2017,UI13,171.13,3700.0,0.0
50,September 2017,UI13,179.78,3700.0,0.0


In [168]:
pay2017["Type"] = pay2017.Name.apply(lambda x : "Monthly" if x =="UI13" else "Hourly")

In [169]:
pay2017.head()

,Month,Name,Total,$,Hourly,Type
0,December 2017,UI13,157.03,3700.00,0.0,Monthly
1,December 2017,UI14,42.03,840.67,20.0,Hourly
2,December 2017,UI15,57.23,1144.67,20.0,Hourly
3,December 2017,UI24,110.00,1870.00,17.0,Hourly
4,December 2017,UI26,28.77,517.80,18.0,Hourly


In [170]:
# droping unneedeed information and renaming columns and rearranging columns
pay2017.drop(columns=["Hourly"], inplace=True)


In [171]:
pay2017.rename(columns={"$": "Total_cost", "Total": "Time_decimal"}, inplace=True)

In [172]:
pay2017.head()

,Month,Name,Time_decimal,Total_cost,Type
0,December 2017,UI13,157.03,3700.00,Monthly
1,December 2017,UI14,42.03,840.67,Hourly
2,December 2017,UI15,57.23,1144.67,Hourly
3,December 2017,UI24,110.00,1870.00,Hourly
4,December 2017,UI26,28.77,517.80,Hourly


In [173]:
pay2017.Month = pay2017.Month.astype('datetime64[ns]')

In [174]:
pay2017 = pay2017[['Month' ,'Name','Time_decimal',"Type", 'Total_cost']]

In [175]:
pay2017 = pay2017.sort_values(["Month", "Name"]).reset_index(drop=True)

### 2) perping pay18-19


In [176]:
pay18_19.dtypes

Unnamed: 0      int64
Month          object
Name           object
Duration       object
Decimal       float64
Type           object
Total         float64
dtype: object

In [177]:
#froping na, renaming columns to uniform and understanble titles, casting date into datestamps, re arranging the rows by date and name, and resetting index 
pay18_19 =( pay18_19.dropna(subset=["Name"]).drop(columns=["Duration", "Unnamed: 0"])
           .rename(columns={"Decimal": "Time_decimal", "Total": "Total_cost"}))
pay18_19.Month = pay18_19.Month.astype('datetime64[ns]')
pay18_19 = pay18_19.sort_values(["Month", "Name"]).reset_index(drop=True)

In [178]:
pay18_19


,Month,Name,Time_decimal,Type,Total_cost
0,2018-02-01,UI10,90.90,Hourly,2272.60
1,2018-02-01,UI13,164.76,Monthly,3700.00
2,2018-02-01,UI14,59.01,Hourly,3180.17
3,2018-02-01,UI15,72.50,Hourly,1450.00
4,2018-02-01,UI18,10.23,Hourly,255.81
...,...,...,...,...,...
470,2019-10-01,UI27,130.58,Monthly,4000.00
471,2019-10-01,UI32,77.66,Hourly,4992.13
472,2019-10-01,UI35,159.17,Hourly,5570.82
473,2019-10-01,UI36,63.67,Hourly,1910.11


### 3) concantating 

In [179]:
pay2017.dtypes

Month           datetime64[ns]
Name                    object
Time_decimal           float64
Type                    object
Total_cost             float64
dtype: object

In [180]:
pay18_19.dtypes

Month           datetime64[ns]
Name                    object
Time_decimal           float64
Type                    object
Total_cost             float64
dtype: object

In [181]:
df_pay = pd.concat([pay2017, pay18_19])

In [182]:
df_pay.reset_index(drop=True, inplace=True)

In [183]:
df_pay

,Month,Name,Time_decimal,Type,Total_cost
0,2017-09-01,UI13,179.78,Monthly,3700.00
1,2017-09-01,UI24,20.00,Hourly,340.00
2,2017-09-01,UI27,68.40,Hourly,1368.00
3,2017-09-01,UI28,33.29,Hourly,599.40
4,2017-09-01,UI32,66.33,Hourly,2056.33
...,...,...,...,...,...
527,2019-10-01,UI27,130.58,Monthly,4000.00
528,2019-10-01,UI32,77.66,Hourly,4992.13
529,2019-10-01,UI35,159.17,Hourly,5570.82
530,2019-10-01,UI36,63.67,Hourly,1910.11


In [184]:
df_pay.dropna()

,Month,Name,Time_decimal,Type,Total_cost
0,2017-09-01,UI13,179.78,Monthly,3700.00
1,2017-09-01,UI24,20.00,Hourly,340.00
2,2017-09-01,UI27,68.40,Hourly,1368.00
3,2017-09-01,UI28,33.29,Hourly,599.40
4,2017-09-01,UI32,66.33,Hourly,2056.33
...,...,...,...,...,...
527,2019-10-01,UI27,130.58,Monthly,4000.00
528,2019-10-01,UI32,77.66,Hourly,4992.13
529,2019-10-01,UI35,159.17,Hourly,5570.82
530,2019-10-01,UI36,63.67,Hourly,1910.11


In [185]:
# generating a price per hour column based on the total time columns and cost columns 
df_pay = df_pay.assign(Cost_per_hour= lambda x : x.Total_cost / x.Time_decimal)

In [186]:
df_pay

,Month,Name,Time_decimal,Type,Total_cost,Cost_per_hour
0,2017-09-01,UI13,179.78,Monthly,3700.00,20.580710
1,2017-09-01,UI24,20.00,Hourly,340.00,17.000000
2,2017-09-01,UI27,68.40,Hourly,1368.00,20.000000
3,2017-09-01,UI28,33.29,Hourly,599.40,18.005407
4,2017-09-01,UI32,66.33,Hourly,2056.33,31.001508
...,...,...,...,...,...,...
527,2019-10-01,UI27,130.58,Monthly,4000.00,30.632562
528,2019-10-01,UI32,77.66,Hourly,4992.13,64.281870
529,2019-10-01,UI35,159.17,Hourly,5570.82,34.999183
530,2019-10-01,UI36,63.67,Hourly,1910.11,30.000157


## Cleaning and merging the IU titles in to the pay database


In [187]:
title.head()

,Unnamed: 0,Name,Position
0,0,UI01,Senior Full-Stack Developer
1,1,UI02,Full-Stack Developer
2,2,UI03,Senior Back End Developer
3,3,UI04,Senior Front End developer
4,4,UI05,Backend Developer


In [188]:
title = title.drop(columns=['Unnamed: 0'])

In [189]:
title.columns

Index(['Name', 'Position'], dtype='object')

In [190]:
title.Position.replace({'Back End Developer ': 'Back End Developer', 'Backend Developer': 'Back End Developer' }, inplace= True)
title.Position= title.Position.str.title()

In [191]:
list(title.Position.unique())


['Senior Full-Stack Developer',
 'Full-Stack Developer',
 'Senior Back End Developer',
 'Senior Front End Developer',
 'Back End Developer',
 'Senior Devops',
 'Devops',
 'Junior Front End Developer',
 'Front End Developer',
 'Quality Assurance',
 'Project Manager / Tech Lead',
 'Junior Back End Developer',
 'Designer']

In [192]:
# the merge
df_pay = df_pay.merge(title, how="left", on="Name")

In [193]:
df_pay.isnull().sum()

Month            0
Name             0
Time_decimal     0
Type             0
Total_cost       0
Cost_per_hour    0
Position         0
dtype: int64

In [194]:
df_pay

,Month,Name,Time_decimal,Type,Total_cost,Cost_per_hour,Position
0,2017-09-01,UI13,179.78,Monthly,3700.00,20.580710,Senior Back End Developer
1,2017-09-01,UI24,20.00,Hourly,340.00,17.000000,Designer
2,2017-09-01,UI27,68.40,Hourly,1368.00,20.000000,Front End Developer
3,2017-09-01,UI28,33.29,Hourly,599.40,18.005407,Quality Assurance
4,2017-09-01,UI32,66.33,Hourly,2056.33,31.001508,Project Manager / Tech Lead
...,...,...,...,...,...,...,...
527,2019-10-01,UI27,130.58,Monthly,4000.00,30.632562,Front End Developer
528,2019-10-01,UI32,77.66,Hourly,4992.13,64.281870,Project Manager / Tech Lead
529,2019-10-01,UI35,159.17,Hourly,5570.82,34.999183,Senior Full-Stack Developer
530,2019-10-01,UI36,63.67,Hourly,1910.11,30.000157,Senior Full-Stack Developer


In [195]:
# now we save this data base as cleaned data base as data_payments
df_pay.to_csv("../cleaned_data_base/data_payments.csv")

In [196]:
# make a new data base ready for a merge 
df_pay_merge = df_pay.drop(columns= ['Time_decimal', 'Total_cost', 'Type']).copy()

In [197]:
df_pay_merge

,Month,Name,Cost_per_hour,Position
0,2017-09-01,UI13,20.580710,Senior Back End Developer
1,2017-09-01,UI24,17.000000,Designer
2,2017-09-01,UI27,20.000000,Front End Developer
3,2017-09-01,UI28,18.005407,Quality Assurance
4,2017-09-01,UI32,31.001508,Project Manager / Tech Lead
...,...,...,...,...
527,2019-10-01,UI27,30.632562,Front End Developer
528,2019-10-01,UI32,64.281870,Project Manager / Tech Lead
529,2019-10-01,UI35,34.999183,Senior Full-Stack Developer
530,2019-10-01,UI36,30.000157,Senior Full-Stack Developer


### what information is need:



# perping the merege with the time entries
End result need to be on each time entry having a cost per hour. be cuae one data set is on a monthly bases and the other is on a daily we need make a dummy colums comprimzed form the UI and the month and do a left join on the tie entry
### 1) cleaning the time entry


In [198]:
# no substantial np.nan in the data base. only 8 in Task description. any anlysis will still be ststiticaly sound
#Filling the the np.nan with empty strings to make 
time_entries.Task_Description.fillna(" ", inplace=True)

In [199]:
time_entries.dtypes

Unnamed: 0           int64
Unnamed: 0.1         int64
User                object
Project             object
Task_cat            object
Task_Description    object
Start date          object
Start time          object
End date            object
End time            object
Duration            object
dtype: object

In [200]:
# Droping unneeded colums: no need for start time and end time when there is a duration.
# we will desicde that the start date will be the date we use. 
#After exploring the data we can see that in most cases the starting date and end date are the same. 
#the othe cases the end date is allways the followinf date. 
#In only few cases the end date lasnds in another month.

time_entries = (time_entries.drop(columns= ['Unnamed: 0', 'Unnamed: 0.1', 'Start time', 'End date', 'End time'])
# renaming the colunms
              .rename(columns={'Start date': 'Date', 'Task_cat': 'Task_Category'}))

In [201]:
time_entries.columns

Index(['User', 'Project', 'Task_Category', 'Task_Description', 'Date',
       'Duration'],
      dtype='object')

In [202]:
time_entries.Duration.value_counts()

01:00:00    308
00:30:00    294
02:00:00    201
04:00:00    158
03:00:00    152
           ... 
13:53:00      1
01:20:16      1
02:40:37      1
04:53:07      1
01:45:51      1
Name: Duration, Length: 10303, dtype: int64

In [203]:
# making a decimal time entry for the duration columns   
time_entries["Time_Decimal"] = (time_entries.Duration.str.split(":")
                              .apply(lambda x: int(x[0]) + int(x[1])/60 + int(x[2])/3600)
                               )

In [204]:
time_entries.head(100)

,User,Project,Task_Category,Task_Description,Date,Duration,Time_Decimal
0,UI13,Revenify,Development,Feedback on budgets index PR and Standup meeti...,2017-09-01,01:17:00,1.283333
1,UI36,WWU,Development,Create graphs in XLSX format from user input,2017-09-01,02:30:00,2.500000
2,UI36,WWU,Development,Create graphs in XLSX format from user input,2017-09-01,00:36:49,0.613611
3,UI38,WWU,Development,Weekly Standup,2017-09-01,00:12:05,0.201389
4,UI36,WWU,Development,Weekly standup,2017-09-01,00:09:07,0.151944
...,...,...,...,...,...,...,...
95,UI36,WWU,DevOps,deployment using EB,2017-09-13,01:29:22,1.489444
96,UI38,WeVape,Development,Change the product list in the Admin panel,2017-09-13,01:30:03,1.500833
97,UI38,Credious,Development,Weekly Standup,2017-09-13,00:13:20,0.222222
98,UI36,WWU,DevOps,deployment using EB,2017-09-13,03:40:45,3.679167


In [205]:
time_entries

,User,Project,Task_Category,Task_Description,Date,Duration,Time_Decimal
0,UI13,Revenify,Development,Feedback on budgets index PR and Standup meeti...,2017-09-01,01:17:00,1.283333
1,UI36,WWU,Development,Create graphs in XLSX format from user input,2017-09-01,02:30:00,2.500000
2,UI36,WWU,Development,Create graphs in XLSX format from user input,2017-09-01,00:36:49,0.613611
3,UI38,WWU,Development,Weekly Standup,2017-09-01,00:12:05,0.201389
4,UI36,WWU,Development,Weekly standup,2017-09-01,00:09:07,0.151944
...,...,...,...,...,...,...,...
25009,UI35,CareBetter,Development,[DB_ADMIN_002] Create user <> admin association,2019-09-28,00:41:22,0.689444
25010,UI20,CandidatePool,Development,Working on adding email and owner to applicant...,2019-09-28,00:20:55,0.348611
25011,UI14,CandidatePool,Development,[FrontEnd] : Add an Alert when The User does n...,2019-09-28,01:02:00,1.033333
25012,UI35,CareBetter,Development,[DB_ADMIN_002] Create user <> admin association,2019-09-28,00:20:59,0.349722


In [206]:
time_entries.dtypes

User                 object
Project              object
Task_Category        object
Task_Description     object
Date                 object
Duration             object
Time_Decimal        float64
dtype: object

### prepering the dummy columns from the date (year and month and User) to merge on
#### 1) time entries

In [207]:
time_entries.Date[0][0:7]

'2017-09'

In [208]:
time_entries.User[0]

'UI13'

In [209]:
time_entries.Date[0][0:7]+time_entries.User[0]

'2017-09UI13'

In [210]:
time_entries = time_entries.assign(dummy= lambda x : x.Date.str[0:7]+x.User) 

In [211]:
time_entries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25014 entries, 0 to 25013
Data columns (total 8 columns):
User                25014 non-null object
Project             25014 non-null object
Task_Category       25014 non-null object
Task_Description    25014 non-null object
Date                25014 non-null object
Duration            25014 non-null object
Time_Decimal        25014 non-null float64
dummy               25014 non-null object
dtypes: float64(1), object(7)
memory usage: 1.5+ MB


#### 2) df_pay_merge

In [212]:
df_pay_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 532 entries, 0 to 531
Data columns (total 4 columns):
Month            532 non-null datetime64[ns]
Name             532 non-null object
Cost_per_hour    532 non-null float64
Position         532 non-null object
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 20.8+ KB


In [213]:
df_pay_merge.Month = df_pay_merge.Month.astype(str)

In [214]:
df_pay_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 532 entries, 0 to 531
Data columns (total 4 columns):
Month            532 non-null object
Name             532 non-null object
Cost_per_hour    532 non-null float64
Position         532 non-null object
dtypes: float64(1), object(3)
memory usage: 20.8+ KB


In [215]:
df_pay_merge = df_pay_merge.assign(dummy= lambda x : x.Month.str[0:7]+x.Name) 

In [216]:
df_pay_merge

,Month,Name,Cost_per_hour,Position,dummy
0,2017-09-01,UI13,20.580710,Senior Back End Developer,2017-09UI13
1,2017-09-01,UI24,17.000000,Designer,2017-09UI24
2,2017-09-01,UI27,20.000000,Front End Developer,2017-09UI27
3,2017-09-01,UI28,18.005407,Quality Assurance,2017-09UI28
4,2017-09-01,UI32,31.001508,Project Manager / Tech Lead,2017-09UI32
...,...,...,...,...,...
527,2019-10-01,UI27,30.632562,Front End Developer,2019-10UI27
528,2019-10-01,UI32,64.281870,Project Manager / Tech Lead,2019-10UI32
529,2019-10-01,UI35,34.999183,Senior Full-Stack Developer,2019-10UI35
530,2019-10-01,UI36,30.000157,Senior Full-Stack Developer,2019-10UI36


In [217]:
df = time_entries.merge(df_pay_merge, how="left",on="dummy")

In [218]:
df

,User,Project,Task_Category,Task_Description,Date,Duration,Time_Decimal,dummy,Month,Name,Cost_per_hour,Position
0,UI13,Revenify,Development,Feedback on budgets index PR and Standup meeti...,2017-09-01,01:17:00,1.283333,2017-09UI13,2017-09-01,UI13,20.580710,Senior Back End Developer
1,UI36,WWU,Development,Create graphs in XLSX format from user input,2017-09-01,02:30:00,2.500000,2017-09UI36,2017-09-01,UI36,24.999437,Senior Full-Stack Developer
2,UI36,WWU,Development,Create graphs in XLSX format from user input,2017-09-01,00:36:49,0.613611,2017-09UI36,2017-09-01,UI36,24.999437,Senior Full-Stack Developer
3,UI38,WWU,Development,Weekly Standup,2017-09-01,00:12:05,0.201389,2017-09UI38,2017-09-01,UI38,25.000000,Project Manager / Tech Lead
4,UI36,WWU,Development,Weekly standup,2017-09-01,00:09:07,0.151944,2017-09UI36,2017-09-01,UI36,24.999437,Senior Full-Stack Developer
...,...,...,...,...,...,...,...,...,...,...,...,...
25009,UI35,CareBetter,Development,[DB_ADMIN_002] Create user <> admin association,2019-09-28,00:41:22,0.689444,2019-09UI35,2019-09-01,UI35,34.997553,Senior Full-Stack Developer
25010,UI20,CandidatePool,Development,Working on adding email and owner to applicant...,2019-09-28,00:20:55,0.348611,2019-09UI20,2019-09-01,UI20,24.261539,Back End Developer
25011,UI14,CandidatePool,Development,[FrontEnd] : Add an Alert when The User does n...,2019-09-28,01:02:00,1.033333,2019-09UI14,2019-09-01,UI14,28.083971,Front End Developer
25012,UI35,CareBetter,Development,[DB_ADMIN_002] Create user <> admin association,2019-09-28,00:20:59,0.349722,2019-09UI35,2019-09-01,UI35,34.997553,Senior Full-Stack Developer


In [219]:
df.isnull().sum()

User                0
Project             0
Task_Category       0
Task_Description    0
Date                0
Duration            0
Time_Decimal        0
dummy               0
Month               2
Name                2
Cost_per_hour       2
Position            2
dtype: int64

In [220]:
df.loc[df.Name.isnull()]
## after merge we have two entriers with np.nan on the cos_per_hour. no abiltiy to do further analsis. 
## droping

,User,Project,Task_Category,Task_Description,Date,Duration,Time_Decimal,dummy,Month,Name,Cost_per_hour,Position
7492,UI16,GlobalMove,Development,Fixing team members URL's on XXX website,2018-07-02,00:55:00,0.916667,2018-07UI16,NaN,NaN,NaN,NaN
7762,UI16,GlobalMove,Development,Checking there are no broken links on team pag...,2018-07-09,00:33:00,0.550000,2018-07UI16,NaN,NaN,NaN,NaN


In [221]:
df.loc[(df.Name == "UI16") & (df.dummy.str.contains("2018-07"))] 

,User,Project,Task_Category,Task_Description,Date,Duration,Time_Decimal,dummy,Month,Name,Cost_per_hour,Position


In [222]:
df.dropna(subset=["Cost_per_hour"], inplace=True)

In [223]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25012 entries, 0 to 25013
Data columns (total 12 columns):
User                25012 non-null object
Project             25012 non-null object
Task_Category       25012 non-null object
Task_Description    25012 non-null object
Date                25012 non-null object
Duration            25012 non-null object
Time_Decimal        25012 non-null float64
dummy               25012 non-null object
Month               25012 non-null object
Name                25012 non-null object
Cost_per_hour       25012 non-null float64
Position            25012 non-null object
dtypes: float64(2), object(10)
memory usage: 2.5+ MB


In [224]:
df.columns

Index(['User', 'Project', 'Task_Category', 'Task_Description', 'Date',
       'Duration', 'Time_Decimal', 'dummy', 'Month', 'Name', 'Cost_per_hour',
       'Position'],
      dtype='object')

In [225]:
#droping redundant and superfluous information
# and casting date information as timestamp
df = df.drop(columns=['dummy', 'Month', 'Name'])
df.Date=df.Date.astype("datetime64[ns]")

In [226]:
# calculating 
df = df.assign(Task_Cost= lambda x : round(x.Time_Decimal * x.Cost_per_hour, 2))

In [227]:
df

,User,Project,Task_Category,Task_Description,Date,Duration,Time_Decimal,Cost_per_hour,Position,Task_Cost
0,UI13,Revenify,Development,Feedback on budgets index PR and Standup meeti...,2017-09-01,01:17:00,1.283333,20.580710,Senior Back End Developer,26.41
1,UI36,WWU,Development,Create graphs in XLSX format from user input,2017-09-01,02:30:00,2.500000,24.999437,Senior Full-Stack Developer,62.50
2,UI36,WWU,Development,Create graphs in XLSX format from user input,2017-09-01,00:36:49,0.613611,24.999437,Senior Full-Stack Developer,15.34
3,UI38,WWU,Development,Weekly Standup,2017-09-01,00:12:05,0.201389,25.000000,Project Manager / Tech Lead,5.03
4,UI36,WWU,Development,Weekly standup,2017-09-01,00:09:07,0.151944,24.999437,Senior Full-Stack Developer,3.80
...,...,...,...,...,...,...,...,...,...,...
25009,UI35,CareBetter,Development,[DB_ADMIN_002] Create user <> admin association,2019-09-28,00:41:22,0.689444,34.997553,Senior Full-Stack Developer,24.13
25010,UI20,CandidatePool,Development,Working on adding email and owner to applicant...,2019-09-28,00:20:55,0.348611,24.261539,Back End Developer,8.46
25011,UI14,CandidatePool,Development,[FrontEnd] : Add an Alert when The User does n...,2019-09-28,01:02:00,1.033333,28.083971,Front End Developer,29.02
25012,UI35,CareBetter,Development,[DB_ADMIN_002] Create user <> admin association,2019-09-28,00:20:59,0.349722,34.997553,Senior Full-Stack Developer,12.24


In [228]:
#df.Duration= pd.to_datetime(df.Duration, format= '%H:%M:%S').dt.time

In [229]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25012 entries, 0 to 25013
Data columns (total 10 columns):
User                25012 non-null object
Project             25012 non-null object
Task_Category       25012 non-null object
Task_Description    25012 non-null object
Date                25012 non-null datetime64[ns]
Duration            25012 non-null object
Time_Decimal        25012 non-null float64
Cost_per_hour       25012 non-null float64
Position            25012 non-null object
Task_Cost           25012 non-null float64
dtypes: datetime64[ns](1), float64(3), object(6)
memory usage: 2.1+ MB


In [231]:
df.to_csv("../cleaned_data_base/Tasks.csv")